In [14]:
import sys
import urllib

from keras.layers import Dense, Flatten

from keras.models import Sequential
from keras.datasets import fashion_mnist
from keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import optimizers

import numpy as np
from PIL import Image
from io import BytesIO

from studio import fs_tracker, model_util, magics

### Load and preprocess the data

In [15]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### Set up learning parameters

In [17]:
batch_size = 128
no_epochs = 10
lr = 0.01

### Define and train model

In [26]:
%%studio_run 

model = Sequential()
model.add(Flatten(input_shape=(28, 28, 1)))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(10, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=lr),
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(
    fs_tracker.get_model_directory() +
    '/checkpoint.{epoch:02d}-{val_loss:.2f}.hdf')

tbcallback = TensorBoard(log_dir=fs_tracker.get_tensorboard_dir(),
                         histogram_freq=0,
                         write_graph=True,
                         write_images=True)

training_stats = model.fit(
    x_train, y_train, validation_data=(
        x_test,
        y_test),
    epochs=no_epochs,
    callbacks=[checkpointer, tbcallback],
    batch_size=batch_size)

Running studio with experiment key 1c20f9bb-43b9-4d4e-a89d-0e579e5dd8d5
Experiment progress can be viewed/shared at:
http://zoo.studio.ml/experiment/1c20f9bb-43b9-4d4e-a89d-0e579e5dd8d5
studio-run --capture=/Users/peter.zhokhov/.studioml/experiments/1c20f9bb-43b9-4d4e-a89d-0e579e5dd8d5/_ns:_ns --force-git --experiment=1c20f9bb-43b9-4d4e-a89d-0e579e5dd8d5 _script.py
Using TensorFlow backend.
2017-10-17 14:05:10.103870: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-10-17 14:05:10.103905: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-10-17 14:05:10.103914: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are av

IOError: close() called during concurrent operation on the same file object.

In [21]:
training_stats.history['loss']

[1.1210452706336975,
 0.65573663806915283,
 0.57085099439620968,
 0.52837221865653994,
 0.50123132295608519,
 0.4823433803399404,
 0.46791895368893943,
 0.45663348863919578,
 0.4474323210398356,
 0.43698937071164451]